In [1]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pendulum
import operator as op

driver = uc.Chrome()
time.sleep(3)
driver.get('https://www.indeed.com/q-usa-jobs.html')
time.sleep(6)

In [17]:
keyword_filter= "chief"
location_filter = "california"
page_count_filter = "1"
salary_filter = "200000.069 $ year"
start_date_posted_filter = "2024-07-01 00:00:00"
end_date_posted_filter = "2025-01-31 23:59:59"
time.sleep(3)

In [3]:
search_job_keyword = driver.find_element(By.XPATH, '//input[@aria-label="search: Job title, keywords, or company"]')
time.sleep(3)
search_job_keyword.click()
time.sleep(3)
actions = ActionChains(driver)
time.sleep(3)
actions.key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL).send_keys(Keys.BACK_SPACE).perform()
time.sleep(3)
search_job_keyword.send_keys(keyword_filter)
time.sleep(3)
search_location_keyword = driver.find_element(By.XPATH, '//input[@aria-label="Edit location"]')
time.sleep(3)
search_location_keyword.click()
time.sleep(3)
search_location_keyword.send_keys(location_filter)
time.sleep(3)
search_button = driver.find_element(By.XPATH, '//button[@class="yosegi-InlineWhatWhere-primaryButton"]')
time.sleep(3)
search_button.submit()
time.sleep(6)

In [4]:
def search_job_indeed():
    global search_job_indeed_list, search_job_indeed_final_list, search_job_indeed_final_list_filter
    search_job_indeed_final_list = []
    time.sleep(3)
    while True:
        try:
            job_indeed_frame = driver.find_elements(By.XPATH, '//div[@id="mosaic-provider-jobcards"]/ul/li/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a')
            time.sleep(3)
            search_job_indeed_list = list(set((job_indeed.get_attribute('href') for job_indeed in job_indeed_frame)))
            time.sleep(3)
            job_indeed_frame[-1].location_once_scrolled_into_view
            time.sleep(6)
        except:
            print('No more job')
            time.sleep(3)
        search_job_indeed_final_list.extend(search_job_indeed_list)
        time.sleep(3)
        search_job_indeed_final_list = list(set(search_job_indeed_final_list))
        time.sleep(3)
        if (len(search_job_indeed_final_list) % 15 == 0 and len(search_job_indeed_final_list) // 15 == int(page_count_filter)) or (len(search_job_indeed_final_list) % 15 != 0 and len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter)):
            break
        try:
            next_page_button = driver.find_elements(By.XPATH, '//nav[@aria-label="pagination"]/ul/li/a')
            time.sleep(3)
            next_page_button[-1].click()
            time.sleep(3)
            yield len(search_job_indeed_final_list)
        except:
            print('No more page')
            time.sleep(3)
            break

generators = search_job_indeed()
for generator in generators:
    print(generator)

In [5]:
if len(search_job_indeed_final_list) % 15 == 0:
    if len(search_job_indeed_final_list) == 0:
        search_job_indeed_final_list_filter = []
    elif len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]
else:
    if len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    elif len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:(int(page_count_filter) - 1) * 15 + len(search_job_indeed_final_list) % 15]
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]

In [6]:
len(search_job_indeed_final_list_filter)

15

In [7]:
def get_job_indeed_description():
        global job_indeed_description_list
        job_indeed_description_list = []
        for _, job_indeed_url in enumerate(search_job_indeed_final_list_filter):
            driver.get(job_indeed_url)
            time.sleep(6)
            try:
                    company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                    time.sleep(3)
                    sub_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[2]/span').text
                    time.sleep(3)
                    if len(re.findall(r'\b\d+\.\d+\b', sub_company)) > 0:
                        total_company = company
                    else:
                        total_company = company + ' ' + sub_company
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                            time.sleep(3)
                    except:
                            try:
                                    time.sleep(3)
                                    total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span').text
                                    time.sleep(3)
                            except:
                                    try:
                                            time.sleep(3)
                                            total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                            time.sleep(3)
                                    except:
                                            try:
                                                    time.sleep(3)
                                                    total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                                    time.sleep(3)
                                            except:
                                                    time.sleep(3)
                                                    total_company = ''
                                                    time.sleep(3)
            try:
                    header = driver.find_element(By.XPATH, '//div[starts-with(@class,"jobsearch-JobInfoHeader-title-container")]/h1/span').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            header = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/span').text
                            time.sleep(3)
                    except:
                            time.sleep(3)
                            header = ''
                            time.sleep(3)
            try:
                    location = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div[2]/div').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/div[2]/div').text
                            time.sleep(3)
                    except:
                            try:
                                time.sleep(3)
                                location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div[2]/div/div/div/div[2]/div').text
                                time.sleep(3)
                            except:
                                    time.sleep(3)
                                    location = ''
                                    time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_context = soup.find_all('script', string=re.compile(r'@context'))[0].getText().strip()
                time.sleep(3)
                standard_date_posted = re.search(r'\"datePosted\":\"([^"]+)\"', script_context).group(1)
                time.sleep(3)
                readable_date_posted = pendulum.parse(standard_date_posted, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                time.sleep(3)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(3)
                    script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                    time.sleep(3)
                    standard_date_posted = re.search(r'\"datePublished\":(\d+)', script_initial_data).group(1)
                    time.sleep(3)
                    readable_date_posted = pendulum.from_timestamp(int(standard_date_posted) / 1000, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                    time.sleep(3)
                except:
                        time.sleep(3)
                        readable_date_posted = ''
                        time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                time.sleep(3)
                salary = re.search(r'\"salaryText\":\"([^"]+)\"', script_initial_data).group(1)
                time.sleep(3)
            except:
                    time.sleep(3)
                    salary = ''
                    time.sleep(3)
            job_indeed_description_list.append((job_indeed_url, total_company, header, salary, location, readable_date_posted))
            yield total_company, header, location, readable_date_posted, salary
generators = get_job_indeed_description()
for generator in generators:
    print(generator)

('Chai', 'Chief of Staff', 'Palo Alto, CA', '2024-09-17 00:39:21', '$205,000 - $275,000 a year')
('LA Philharmonic', 'Chief Philanthropy Officer', 'Los Angeles, CA', '2024-12-02 22:58:01', '$400,000 - $450,000 a year')
('Proper Hospitality LLC', 'Chief of Staff', '225 Arizona Avenue, Santa Monica, CA 90401', '2025-01-10 08:58:04', '$225,000 - $275,000 a year')
('City of Fountain Valley, CA', 'Chief of Police', 'Fountain Valley, CA', '2024-07-05 21:44:48', '$218,220.88 - $265,248.85 a year')
('California State University', 'Vice President of Equity and Inclusion & Chief Diversity Officer', '18111 Nordhoff St, Northridge, CA 91330', '2024-11-12 20:49:11', '$245,000 - $255,000 a year')
('Cella', 'CHIEF OF STAFF- CMO', 'San Francisco, CA', '2024-11-13 15:38:08', '$105 - $110 an hour')
('UCSF Health', 'Chief Dosimetrist', 'San Francisco, CA', '2024-04-30 22:29:54', '')
('LPL Financial', 'SVP, Chief of Staff', '4707 Executive Dr, San Diego, CA 92121', '2024-12-23 17:37:01', '$167,500 - $279,

In [8]:
len(job_indeed_description_list)

15

In [9]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('credentials.json')
spreadsheet_service = build('sheets', 'v4', credentials=credentials)

SPREADSHEET_ID = '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg'
RANGE_NAME = 'Sheet1'
sheet = spreadsheet_service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
max_cols = max(len(row) for row in values)
values = [row + [''] * (max_cols - len(row)) for row in values]
df = pd.DataFrame(values[1:], columns=values[0])
df = df.fillna('')

In [10]:
cleared_values = {
    "requests": [
        {
            "updateCells": {
                "range": {
                    "sheetId": 0,
                    "startRowIndex": 1,
                    "endRowIndex": None,
                    "startColumnIndex": 0,
                    "endColumnIndex": 6
                },
                "fields": "userEnteredValue"
            }
        }
    ]
}
spreadsheet_service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=cleared_values).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'replies': [{}]}

In [36]:
df1 = pd.DataFrame(job_indeed_description_list, columns=df.columns).__deepcopy__()
df1

,Link,Company,Title,Salary,Location,Date posted
0,https://www.indeed.com/rc/clk?jk=2b5742e906877...,Chai,Chief of Staff,"$205,000 - $275,000 a year","Palo Alto, CA",2024-09-17 00:39:21
1,https://www.indeed.com/rc/clk?jk=68fec70c777f3...,LA Philharmonic,Chief Philanthropy Officer,"$400,000 - $450,000 a year","Los Angeles, CA",2024-12-02 22:58:01
2,https://www.indeed.com/rc/clk?jk=8ccc1e500fb3b...,Proper Hospitality LLC,Chief of Staff,"$225,000 - $275,000 a year","225 Arizona Avenue, Santa Monica, CA 90401",2025-01-10 08:58:04
3,https://www.indeed.com/rc/clk?jk=afbb4c1dbef98...,"City of Fountain Valley, CA",Chief of Police,"$218,220.88 - $265,248.85 a year","Fountain Valley, CA",2024-07-05 21:44:48
4,https://www.indeed.com/rc/clk?jk=6eeebde66863d...,California State University,Vice President of Equity and Inclusion & Chief...,"$245,000 - $255,000 a year","18111 Nordhoff St, Northridge, CA 91330",2024-11-12 20:49:11
5,https://www.indeed.com/rc/clk?jk=f0f00697ee9cf...,Cella,CHIEF OF STAFF- CMO,$105 - $110 an hour,"San Francisco, CA",2024-11-13 15:38:08
6,https://www.indeed.com/rc/clk?jk=8077caa1117c6...,UCSF Health,Chief Dosimetrist,,"San Francisco, CA",2024-04-30 22:29:54
7,https://www.indeed.com/rc/clk?jk=4aef59496e15b...,LPL Financial,"SVP, Chief of Staff","$167,500 - $279,100 a year","4707 Executive Dr, San Diego, CA 92121",2024-12-23 17:37:01
8,https://www.indeed.com/rc/clk?jk=b1551da0fea72...,Kaiser Permanente,Associate Chief Administrative Officer,"$222,300 - $287,650 a year","Vacaville, CA",2024-11-12 16:01:59
9,https://www.indeed.com/rc/clk?jk=1e0d20a749623...,Deep 6 AI,Chief Operating Officer,"$220,000 - $260,000 a year","Pasadena, CA",2024-11-12 05:10:08


In [38]:
def implement_salary_filter():
    global implement_filter_list
    implement_filter_list = []
    salary_filter_tuple = (salary_filter.split(' ')[0], salary_filter.split(' ')[1], salary_filter.split(' ')[2])
    for index, row in df1.iterrows():
        if op.contains(df1.at[index, 'Salary'], salary_filter_tuple[1]) and op.contains(df1.at[index, 'Salary'], salary_filter_tuple[2]):
            if '.' in df1.at[index, 'Salary']:
                range_salary_tuple = tuple(float(salary.replace(',', '')) for salary in re.findall(r"([\d,\.]+)", df1.iloc[index]['Salary']))
            else:
                range_salary_tuple = tuple(int(salary.replace(',', '')) for salary in re.findall(r"([\d,]+)", df1.iloc[index]['Salary']))
            if len(range_salary_tuple) == 2 and (float(salary_filter_tuple[0]) >= range_salary_tuple[0] and float(salary_filter_tuple[0]) <= range_salary_tuple[1]):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) <= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'up to')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) >= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'from')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) == range_salary_tuple[0] and (not op.contains(df1.iloc[index]['Salary'].lower(), 'up to') and not op.contains(df1.iloc[index]['Salary'].lower(), 'from'))):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
    yield from implement_filter_list
salary_generators = implement_salary_filter()
for generator in salary_generators:
    print(generator)

('https://www.indeed.com/rc/clk?jk=4aef59496e15bed8&bb=bAC0RrAkc1-GwjEisDuR6T4Dl4Jh6eYmPg-vTOAnFj4nlydnrKWczuTfQu4FCj7JunqFNNzdRo3TNRyNpHIrjr2-oGieRlKTehCg8yjUt9HEu5hKRn7llQ%3D%3D&xkcb=SoDc67M33r3CyHQVnx0PbzkdCdPP&fccid=bba3b2578c9adb92&vjs=3', 'LPL Financial', 'SVP, Chief of Staff', '$167,500 - $279,100 a year', '4707 Executive Dr, San Diego, CA 92121', '2024-12-23 17:37:01')
('https://www.indeed.com/rc/clk?jk=747605dc022c591c&bb=bAC0RrAkc1-GwjEisDuR6aiulNzSeY3Z3IPi3t2uP3Z0cKyrtOAjP8KFdFOkuhgkyMA_ZGBrkUy8xz9KrkFH6QU1pJtNYYabKD3K3r5HvC9Z9oD9WeL32D18dcADjCSO&xkcb=SoCo67M33r3CyHQVnx0DbzkdCdPP&fccid=7d28c1874e2d22be&vjs=3', 'Bill.com', 'Chief of Staff', '$197,600 - $247,500 a year', 'San Jose, CA', '2025-01-07 23:07:29')
('https://www.indeed.com/rc/clk?jk=f623dbbc2b80f8c8&bb=bAC0RrAkc1-GwjEisDuR6ZKBvGRlp2cPwPCruq0De73_QbWADjM1irn9cW9opMRnb5X4qZ03xl-TEIphVQoM5c4O65-GDGKAhtluw4W1DkDCtNT7EAXcuPH1XQlXdRX-&xkcb=SoB767M33r3CyHQVnx0KbzkdCdPP&fccid=e6d4ba9e2cfe7902&vjs=3', 'Atlassian', 'Chief of 

In [39]:
df2 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()

In [42]:
def implement_date_posted_filter():
    implement_filter_list = []
    for row in df2.itertuples(index=False):
        if pendulum.parse(start_date_posted_filter, tz='Asia/Ho_Chi_Minh') <= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh') and pendulum.parse(end_date_posted_filter, tz='Asia/Ho_Chi_Minh') >= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh'):
            implement_filter_list.append((row.Link, row.Company, row.Title, row.Salary, row.Location, row._5))
    yield from implement_filter_list
date_posted_generators = implement_date_posted_filter()
for generator in date_posted_generators:
    print(generator)
implement_filter_list = list(set(implement_filter_list))

('https://www.indeed.com/rc/clk?jk=4aef59496e15bed8&bb=bAC0RrAkc1-GwjEisDuR6T4Dl4Jh6eYmPg-vTOAnFj4nlydnrKWczuTfQu4FCj7JunqFNNzdRo3TNRyNpHIrjr2-oGieRlKTehCg8yjUt9HEu5hKRn7llQ%3D%3D&xkcb=SoDc67M33r3CyHQVnx0PbzkdCdPP&fccid=bba3b2578c9adb92&vjs=3', 'LPL Financial', 'SVP, Chief of Staff', '$167,500 - $279,100 a year', '4707 Executive Dr, San Diego, CA 92121', '2024-12-23 17:37:01')
('https://www.indeed.com/rc/clk?jk=747605dc022c591c&bb=bAC0RrAkc1-GwjEisDuR6aiulNzSeY3Z3IPi3t2uP3Z0cKyrtOAjP8KFdFOkuhgkyMA_ZGBrkUy8xz9KrkFH6QU1pJtNYYabKD3K3r5HvC9Z9oD9WeL32D18dcADjCSO&xkcb=SoCo67M33r3CyHQVnx0DbzkdCdPP&fccid=7d28c1874e2d22be&vjs=3', 'Bill.com', 'Chief of Staff', '$197,600 - $247,500 a year', 'San Jose, CA', '2025-01-07 23:07:29')
('https://www.indeed.com/rc/clk?jk=f623dbbc2b80f8c8&bb=bAC0RrAkc1-GwjEisDuR6ZKBvGRlp2cPwPCruq0De73_QbWADjM1irn9cW9opMRnb5X4qZ03xl-TEIphVQoM5c4O65-GDGKAhtluw4W1DkDCtNT7EAXcuPH1XQlXdRX-&xkcb=SoB767M33r3CyHQVnx0KbzkdCdPP&fccid=e6d4ba9e2cfe7902&vjs=3', 'Atlassian', 'Chief of 

In [43]:
df3 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()
updated_values = [df3.columns.tolist()] + df3.values.tolist()
body = {'values': updated_values}
spreadsheet_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
    valueInputOption='RAW', body=body).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'updatedRange': 'Sheet1!A1:F4',
 'updatedRows': 4,
 'updatedColumns': 6,
 'updatedCells': 24}